### This notebook will be used mainly for the IBM Course's Applied Data Science Capstone Project

In [2]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import random
#for images
from IPython.display import Image 
from IPython.core.display import HTML 
from IPython.display import display_html

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

#geopy missing

Solving environment: ...working... done

# All requested packages already installed.



In [3]:
!pip install geopy
from geopy.geocoders import Nominatim

In [4]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [5]:
#creating a soup object with the url
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url, 'html.parser')

In [6]:
#we create the dataframe
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [8]:
df.shape

(103, 3)

In [7]:
#getting the geospatial data
coord = pd.read_csv(r'F:\Users\GIUSEPPE\Documents\PyFiles\Geospatial_Coordinates.csv')
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
merged_df = pd.merge(left = df, right = coord, left_on= "PostalCode", right_on = 'Postal Code')
merged_df = merged_df.drop(columns = ['PostalCode'])
merged_df.head()

,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494


## Now we will cluster the Neighborhoods which include Toronto in their borough.

In [12]:
df2 = merged_df[merged_df['Borough'].str.contains('Toronto', regex = False)]
df2.head()

,Borough,Neighborhood,Postal Code,Latitude,Longitude
2,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
9,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937
15,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418
19,East Toronto,The Beaches,M4E,43.676357,-79.293031
20,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306


## We will now use Folium to visualize the Neighbourhoods.

In [18]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighborhood in zip(df2['Latitude'],df2['Longitude'],df2['Borough'],df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='black',
    fill=True,
    fill_color='red',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

# DISCLAIMER: You might not be able to see the map on github. There will be pictures of the maps on the README.

## Now we use k-means clustering to cluster the neighborhoods

In [22]:
k = 5
toronto_clustering = df2.drop(['Postal Code', 'Borough', 'Neighborhood'], 1)
kmeans = KMeans(n_clusters = k, random_state = 1).fit(toronto_clustering)
kmeans.labels_[0:10]
df2.insert(0, 'Cluster Labels', kmeans.labels_)

In [23]:
df2

,Cluster Labels,Borough,Neighborhood,Postal Code,Latitude,Longitude
2,0,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
9,0,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937
15,0,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418
19,3,East Toronto,The Beaches,M4E,43.676357,-79.293031
20,0,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306
24,0,Downtown Toronto,Central Bay Street,M5G,43.657952,-79.387383
25,2,Downtown Toronto,Christie,M6G,43.669542,-79.422564
30,0,Downtown Toronto,"Richmond, Adelaide, King",M5H,43.650571,-79.384568
31,4,West Toronto,"Dufferin, Dovercourt Village",M6H,43.669005,-79.442259
35,3,East York/East Toronto,The Danforth East,M4J,43.685347,-79.338106


### We crate again the map using folium, this time with color coded clusters.

In [25]:
#map creation

map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

#setting color scheme for clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#adding markers
for lat, lon, poc, cluster in zip(df2['Latitude'], df2['Longitude'],df2['Neighborhood'], df2['Cluster Labels']):
    label = folium.Popup(str(poc) + ' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters